# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [23]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [24]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [25]:
full_data_rows_list = [] 
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
                
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

print(len(full_data_rows_list))
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


#### check the number of rows in your csv file

In [26]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [27]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [28]:
 try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [29]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Create table app_history:
Table Name: app_history <br>
column 1: artist <br>
column 2: song_title <br>
column 3: song_length <br>
column 4: session Id <br>
column 5: itemInSession <br>
Primary Key(sessionId, itemInSession) <br>

In [30]:
query = "CREATE TABLE IF NOT EXISTS app_history "
query = query + "(artist text, song_title text, song_length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession));"
try:
    session.execute(query)
except Exception as e:
    print(e)               

### Read cvs and Insert Data into Table app_history

In [31]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_history (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted 

Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


In [32]:
query = "SELECT artist, song_title, song_length FROM app_history WHERE sessionId = 338 AND itemInSession = 4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Create table users_history:
Table Name: users_history <br>
column 1: artist <br>
column 2: song_title <br>
column 3: user_firstname <br>
column 4: user_lastname <br>
column 5: userId <br> 
column 6: sessionId <br>
column 7: itemInSession <br>
Primary Key((userId, sessionId), itemInSession) <br>

In [33]:
query = "CREATE TABLE IF NOT EXISTS users_history "
query = query + "(artist text, song_title text, user_firstname text, user_lastname text, userId int, sessionId int, itemInSession int, PRIMARY KEY ((userId, sessionId), itemInSession));"
try:
    session.execute(query)
except Exception as e:
    print(e)


### Read cvs and Insert Data into Table users_history

In [34]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO users_history (userId, sessionId, itemInSession, artist, song_title, user_firstname, user_lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data 

Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [35]:
query = "SELECT artist, song_title, user_firstname, user_lastname, iteminsession FROM users_history WHERE userid = 10 AND sessionid = 182 ORDER BY itemInSession ASC;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.user_firstname, row.user_lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Create table users_music:
Table Name: users_music <br>
column 1: artist <br>
column 2: song_title <br>
column 3: user_firstname <br>
column 4: user_lastname <br>
column 5: userId <br> 
Primary Key(song_title, userId) <br>

In [36]:
query = "CREATE TABLE IF NOT EXISTS users_music "
query = query + "(artist text, song_title text, user_firstname text, user_lastname text, userId int, PRIMARY KEY (song_title, userId));"
try:
    session.execute(query)
except Exception as e:
    print(e)                  

### Read cvs and Insert Data into Table users_music

In [37]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO users_music (song_title, userId, artist, user_firstname, user_lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[0],line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted 

Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [38]:
query = "SELECT user_firstname, user_lastname FROM users_music WHERE song_title = 'All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_firstname, row.user_lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [40]:
table_list = ["app_history", 'users_history', 'users_music']
query = "drop table "
try:
    for i in table_list:
        session.execute(query + i)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [41]:
session.shutdown()
cluster.shutdown()